# COVID-19 Diagnosis Expert System

This expert system uses CLIPS (clipspy) for rule-based reasoning and tkinter for the GUI interface.

## Install Required Libraries
Run this cell first if you haven't installed the required packages:

In [ ]:
# Uncomment and run if packages are not installed
# !pip install clipspy
# tkinter comes pre-installed with Python

## Import Required Libraries

In [3]:
import clips
import tkinter as tk
from tkinter import ttk, messagebox, scrolledtext
from typing import Dict, List

## Define CLIPS Expert System Rules

We'll create a rule-based system with multiple rules for COVID-19 diagnosis based on symptoms and exposure.

In [5]:
class COVID19ExpertSystem:
    def __init__(self):
        self.env = clips.Environment()
        self.define_templates()
        self.define_rules()
        
    def define_templates(self):
        """Define CLIPS templates for facts"""
        # Template for patient symptoms
        self.env.build("""
            (deftemplate patient
                (slot fever (type SYMBOL) (allowed-symbols yes no))
                (slot cough (type SYMBOL) (allowed-symbols yes no))
                (slot breathing-difficulty (type SYMBOL) (allowed-symbols yes no))
                (slot fatigue (type SYMBOL) (allowed-symbols yes no))
                (slot loss-of-taste-smell (type SYMBOL) (allowed-symbols yes no))
                (slot sore-throat (type SYMBOL) (allowed-symbols yes no))
                (slot body-aches (type SYMBOL) (allowed-symbols yes no))
                (slot recent-travel (type SYMBOL) (allowed-symbols yes no))
                (slot contact-with-positive (type SYMBOL) (allowed-symbols yes no)))
        """)
        
        # Template for diagnosis result
        self.env.build("""
            (deftemplate diagnosis
                (slot risk-level (type SYMBOL))
                (slot recommendation (type STRING))
                (slot confidence (type STRING)))
        """)
    
    def define_rules(self):
        """Define expert system rules for COVID-19 diagnosis"""
        
        # Rule 1: High Risk - Classic COVID-19 symptoms with key indicators
        self.env.build("""
            (defrule high-risk-covid
                "Identifies high risk COVID-19 cases based on classic symptoms"
                (patient 
                    (fever yes)
                    (cough yes)
                    (loss-of-taste-smell yes))
                =>
                (assert (diagnosis 
                    (risk-level HIGH)
                    (recommendation "URGENT: Get tested for COVID-19 immediately. Isolate yourself from others. Loss of taste/smell with fever and cough are strong indicators of COVID-19. Contact your healthcare provider.")
                    (confidence "High - 85-90%"))))
        """)
        
        # Rule 2: High Risk - Severe symptoms
        self.env.build("""
            (defrule severe-symptoms
                "Identifies severe cases requiring immediate medical attention"
                (patient 
                    (breathing-difficulty yes)
                    (fever yes))
                =>
                (assert (diagnosis 
                    (risk-level SEVERE)
                    (recommendation "EMERGENCY: Seek immediate medical attention! Breathing difficulty with fever may indicate severe COVID-19 or other serious respiratory illness. Call emergency services or go to ER.")
                    (confidence "Critical - Immediate Action Required"))))
        """)
        
        # Rule 3: Medium Risk - Multiple symptoms with exposure
        self.env.build("""
            (defrule medium-risk-with-exposure
                "Identifies medium risk cases with exposure history"
                (patient 
                    (fever yes)
                    (contact-with-positive yes))
                =>
                (assert (diagnosis 
                    (risk-level MEDIUM)
                    (recommendation "CAUTION: You have fever and recent contact with a COVID-19 positive person. Get tested as soon as possible. Self-isolate and monitor symptoms closely.")
                    (confidence "Medium-High - 70-80%"))))
        """)
        
        # Rule 4: Medium Risk - Travel with symptoms
        self.env.build("""
            (defrule medium-risk-travel
                "Identifies medium risk from travel with symptoms"
                (patient 
                    (recent-travel yes)
                    (cough yes)
                    (fatigue yes))
                =>
                (assert (diagnosis 
                    (risk-level MEDIUM)
                    (recommendation "CAUTION: Recent travel with respiratory symptoms. Get tested for COVID-19. Self-isolate until test results are available.")
                    (confidence "Medium - 60-70%"))))
        """)
        
        # Rule 5: Low Risk - Mild symptoms only
        self.env.build("""
            (defrule low-risk-mild
                "Identifies low risk cases with only mild symptoms"
                (patient 
                    (fever no)
                    (breathing-difficulty no)
                    (loss-of-taste-smell no)
                    (contact-with-positive no))
                (or (patient (sore-throat yes))
                    (patient (body-aches yes)))
                =>
                (assert (diagnosis 
                    (risk-level LOW)
                    (recommendation "LOW RISK: Your symptoms suggest a mild illness, possibly common cold or flu. Monitor symptoms. If they worsen or you develop fever, get tested. Rest and stay hydrated.")
                    (confidence "Low - 20-30%"))))
        """)
        
        # Rule 6: Very Low Risk - No significant symptoms
        self.env.build("""
            (defrule very-low-risk
                "Identifies very low risk cases"
                (patient 
                    (fever no)
                    (cough no)
                    (breathing-difficulty no)
                    (loss-of-taste-smell no)
                    (contact-with-positive no))
                =>
                (assert (diagnosis 
                    (risk-level VERY-LOW)
                    (recommendation "VERY LOW RISK: No significant COVID-19 symptoms detected. Continue following preventive measures: wear masks, maintain social distancing, and practice good hygiene.")
                    (confidence "Very Low - <10%"))))
        """)
    
    def diagnose(self, symptoms: Dict[str, str]) -> List[Dict]:
        """Run diagnosis based on patient symptoms"""
        self.env.reset()
        
        # Assert patient facts
        fact_string = f"""(patient 
            (fever {symptoms.get('fever', 'no')})
            (cough {symptoms.get('cough', 'no')})
            (breathing-difficulty {symptoms.get('breathing-difficulty', 'no')})
            (fatigue {symptoms.get('fatigue', 'no')})
            (loss-of-taste-smell {symptoms.get('loss-of-taste-smell', 'no')})
            (sore-throat {symptoms.get('sore-throat', 'no')})
            (body-aches {symptoms.get('body-aches', 'no')})
            (recent-travel {symptoms.get('recent-travel', 'no')})
            (contact-with-positive {symptoms.get('contact-with-positive', 'no')}))"""
        
        self.env.assert_string(fact_string)
        
        # Run the expert system
        self.env.run()
        
        # Collect all diagnosis results
        results = []
        for fact in self.env.facts():
            if fact.template.name == 'diagnosis':
                results.append({
                    'risk_level': str(fact['risk-level']),
                    'recommendation': str(fact['recommendation']),
                    'confidence': str(fact['confidence'])
                })
        
        return results

## Create Tkinter GUI Interface

In [6]:
class COVID19DiagnosisGUI:
    def __init__(self, master):
        self.master = master
        self.master.title("COVID-19 Diagnosis Expert System")
        self.master.geometry("700x800")
        self.master.resizable(True, True)
        
        # Initialize expert system
        self.expert_system = COVID19ExpertSystem()
        
        # Symptom variables
        self.symptom_vars = {}
        
        self.create_widgets()
    
    def create_widgets(self):
        """Create all GUI widgets"""
        # Title
        title_frame = tk.Frame(self.master, bg='#2c3e50', pady=15)
        title_frame.pack(fill='x')
        
        title_label = tk.Label(
            title_frame, 
            text="COVID-19 Diagnosis Expert System",
            font=('Arial', 18, 'bold'),
            bg='#2c3e50',
            fg='white'
        )
        title_label.pack()
        
        subtitle = tk.Label(
            title_frame,
            text="Powered by CLIPS Expert System",
            font=('Arial', 10),
            bg='#2c3e50',
            fg='#ecf0f1'
        )
        subtitle.pack()
        
        # Main container
        main_frame = tk.Frame(self.master, padx=20, pady=20)
        main_frame.pack(fill='both', expand=True)
        
        # Instructions
        instruction_label = tk.Label(
            main_frame,
            text="Please select all symptoms and conditions that apply:",
            font=('Arial', 11, 'bold'),
            justify='left'
        )
        instruction_label.pack(anchor='w', pady=(0, 10))
        
        # Symptoms frame with scrollbar
        canvas_frame = tk.Frame(main_frame)
        canvas_frame.pack(fill='both', expand=True)
        
        # Symptoms section
        symptoms_frame = tk.LabelFrame(
            canvas_frame,
            text="Symptoms & Exposure History",
            font=('Arial', 10, 'bold'),
            padx=15,
            pady=15
        )
        symptoms_frame.pack(fill='both', expand=True, pady=(0, 10))
        
        # Define symptoms
        symptoms = [
            ('fever', 'Fever (temperature > 100.4°F / 38°C)'),
            ('cough', 'Dry Cough'),
            ('breathing-difficulty', 'Difficulty Breathing / Shortness of Breath'),
            ('fatigue', 'Fatigue / Tiredness'),
            ('loss-of-taste-smell', 'Loss of Taste or Smell'),
            ('sore-throat', 'Sore Throat'),
            ('body-aches', 'Body Aches / Muscle Pain'),
            ('recent-travel', 'Recent Travel (last 14 days)'),
            ('contact-with-positive', 'Contact with COVID-19 Positive Person')
        ]
        
        # Create checkboxes for each symptom
        for i, (key, label) in enumerate(symptoms):
            var = tk.BooleanVar()
            self.symptom_vars[key] = var
            
            cb = tk.Checkbutton(
                symptoms_frame,
                text=label,
                variable=var,
                font=('Arial', 10),
                anchor='w'
            )
            cb.pack(anchor='w', pady=5)
        
        # Buttons frame
        button_frame = tk.Frame(main_frame)
        button_frame.pack(pady=15)
        
        diagnose_btn = tk.Button(
            button_frame,
            text="Get Diagnosis",
            command=self.perform_diagnosis,
            bg='#27ae60',
            fg='white',
            font=('Arial', 12, 'bold'),
            padx=30,
            pady=10,
            cursor='hand2'
        )
        diagnose_btn.pack(side='left', padx=5)
        
        reset_btn = tk.Button(
            button_frame,
            text="Reset",
            command=self.reset_form,
            bg='#e74c3c',
            fg='white',
            font=('Arial', 12, 'bold'),
            padx=30,
            pady=10,
            cursor='hand2'
        )
        reset_btn.pack(side='left', padx=5)
        
        # Results frame
        results_frame = tk.LabelFrame(
            main_frame,
            text="Diagnosis Results",
            font=('Arial', 10, 'bold'),
            padx=15,
            pady=15
        )
        results_frame.pack(fill='both', expand=True)
        
        self.results_text = scrolledtext.ScrolledText(
            results_frame,
            height=10,
            font=('Arial', 10),
            wrap='word',
            state='disabled'
        )
        self.results_text.pack(fill='both', expand=True)
        
        # Disclaimer
        disclaimer = tk.Label(
            main_frame,
            text="⚠ DISCLAIMER: This is an educational tool only. Not a substitute for professional medical advice.",
            font=('Arial', 9, 'italic'),
            fg='#e74c3c',
            wraplength=650,
            justify='center'
        )
        disclaimer.pack(pady=10)
    
    def perform_diagnosis(self):
        """Perform diagnosis using the expert system"""
        # Collect symptoms
        symptoms = {}
        for key, var in self.symptom_vars.items():
            symptoms[key] = 'yes' if var.get() else 'no'
        
        # Check if any symptom is selected
        if all(v == 'no' for v in symptoms.values()):
            messagebox.showwarning(
                "No Symptoms Selected",
                "Please select at least one symptom or condition to get a diagnosis."
            )
            return
        
        # Get diagnosis
        results = self.expert_system.diagnose(symptoms)
        
        # Display results
        self.display_results(results, symptoms)
    
    def display_results(self, results, symptoms):
        """Display diagnosis results"""
        self.results_text.config(state='normal')
        self.results_text.delete('1.0', tk.END)
        
        # Show selected symptoms
        self.results_text.insert(tk.END, "SELECTED SYMPTOMS/CONDITIONS:\n", 'header')
        self.results_text.insert(tk.END, "=" * 60 + "\n\n")
        
        selected = [k.replace('-', ' ').title() for k, v in symptoms.items() if v == 'yes']
        for symptom in selected:
            self.results_text.insert(tk.END, f"  • {symptom}\n")
        
        self.results_text.insert(tk.END, "\n" + "=" * 60 + "\n\n")
        
        if results:
            # Sort by severity
            severity_order = {'SEVERE': 0, 'HIGH': 1, 'MEDIUM': 2, 'LOW': 3, 'VERY-LOW': 4}
            results.sort(key=lambda x: severity_order.get(x['risk_level'], 999))
            
            for i, result in enumerate(results, 1):
                self.results_text.insert(tk.END, f"DIAGNOSIS #{i}:\n", 'subheader')
                self.results_text.insert(tk.END, f"Risk Level: {result['risk_level']}\n", 'risk')
                self.results_text.insert(tk.END, f"Confidence: {result['confidence']}\n\n")
                self.results_text.insert(tk.END, f"Recommendation:\n{result['recommendation']}\n\n")
                self.results_text.insert(tk.END, "=" * 60 + "\n\n")
        else:
            self.results_text.insert(
                tk.END,
                "No specific diagnosis could be determined based on the symptoms provided.\n"
                "Please consult a healthcare professional for proper evaluation."
            )
        
        # Configure text tags for styling
        self.results_text.tag_config('header', font=('Arial', 11, 'bold'))
        self.results_text.tag_config('subheader', font=('Arial', 10, 'bold'))
        self.results_text.tag_config('risk', foreground='#e74c3c', font=('Arial', 10, 'bold'))
        
        self.results_text.config(state='disabled')
    
    def reset_form(self):
        """Reset all form fields"""
        for var in self.symptom_vars.values():
            var.set(False)
        
        self.results_text.config(state='normal')
        self.results_text.delete('1.0', tk.END)
        self.results_text.config(state='disabled')

## Run the Application

In [8]:
# Create and run the GUI application
def main():
    root = tk.Tk()
    app = COVID19DiagnosisGUI(root)
    root.mainloop()

if __name__ == "__main__":
    main()

## Test the Expert System (Optional - Console Mode)

In [ ]:
# Test the expert system without GUI
def test_expert_system():
    expert_system = COVID19ExpertSystem()
    
    # Test Case 1: High Risk
    print("\n" + "="*70)
    print("TEST CASE 1: High Risk - Classic COVID symptoms")
    print("="*70)
    test_symptoms_1 = {
        'fever': 'yes',
        'cough': 'yes',
        'loss-of-taste-smell': 'yes',
        'fatigue': 'yes'
    }
    results = expert_system.diagnose(test_symptoms_1)
    for result in results:
        print(f"\nRisk Level: {result['risk_level']}")
        print(f"Confidence: {result['confidence']}")
        print(f"Recommendation: {result['recommendation']}")
    
    # Test Case 2: Severe
    print("\n" + "="*70)
    print("TEST CASE 2: Severe - Breathing difficulty with fever")
    print("="*70)
    test_symptoms_2 = {
        'fever': 'yes',
        'breathing-difficulty': 'yes',
        'cough': 'yes'
    }
    results = expert_system.diagnose(test_symptoms_2)
    for result in results:
        print(f"\nRisk Level: {result['risk_level']}")
        print(f"Confidence: {result['confidence']}")
        print(f"Recommendation: {result['recommendation']}")
    
    # Test Case 3: Low Risk
    print("\n" + "="*70)
    print("TEST CASE 3: Low Risk - Only mild symptoms")
    print("="*70)
    test_symptoms_3 = {
        'sore-throat': 'yes',
        'body-aches': 'yes'
    }
    results = expert_system.diagnose(test_symptoms_3)
    for result in results:
        print(f"\nRisk Level: {result['risk_level']}")
        print(f"Confidence: {result['confidence']}")
        print(f"Recommendation: {result['recommendation']}")

# Uncomment to run tests
# test_expert_system()

## Rules Summary

### Expert System Rules Implemented:

1. **High Risk COVID Rule**: Identifies high-risk cases with classic COVID-19 symptoms (fever + cough + loss of taste/smell)

2. **Severe Symptoms Rule**: Identifies critical cases requiring immediate medical attention (breathing difficulty + fever)

3. **Medium Risk with Exposure Rule**: Identifies medium-risk cases with exposure history (fever + contact with positive person)

4. **Medium Risk Travel Rule**: Identifies medium-risk cases from recent travel with symptoms (travel + cough + fatigue)

5. **Low Risk Mild Symptoms Rule**: Identifies low-risk cases with only mild symptoms and no key indicators

6. **Very Low Risk Rule**: Identifies very low-risk cases with no significant COVID-19 symptoms

### Features:
- Forward chaining rule-based inference using CLIPS
- Multiple rules can fire simultaneously for comprehensive diagnosis
- Risk stratification: SEVERE, HIGH, MEDIUM, LOW, VERY-LOW
- Confidence levels provided for each diagnosis
- User-friendly tkinter GUI interface
- Real-time symptom assessment

### Note:
This is an educational expert system and should not be used for actual medical diagnosis. Always consult healthcare professionals for medical advice.